In [1]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1

from tqdm.notebook import tqdm
import wandb

from FNN import *

wandb.login()

env: PYTORCH_ENABLE_MPS_FALLBACK=1


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [2]:
DEVICE = "cpu"

In [3]:
def get_dense_model(input_size: int,
                    hidden_sizes: list[int],
                    output_size: int,
                    activation_func: callable):
    layers = []

    for size in hidden_sizes:
        layers.append(torch.nn.Linear(input_size, size))
        layers.append(activation_func())
        input_size = size

    layers.append(torch.nn.Linear(input_size, output_size))

    return torch.nn.Sequential(*layers)

In [4]:
WindDataset.init(0.1)

train = WindDataset("train")
validation = WindDataset("validation")
test = WindDataset("test")

del WindDataset.data

In [5]:
INPUT_SIZE = 15
OUTPUT_SIZE = 1

EPOCHS = 30

LOSS_FUNC = torch.nn.MSELoss

In [6]:
def evaluate_one_epoch(model, epoch, dl):
    mse = 0
    mae = 0

    with torch.no_grad():
        for inputs, targets in dl:
            prediction = model(inputs).squeeze()

            mse += torch.nn.functional.mse_loss(prediction, targets)
            mae += torch.nn.functional.l1_loss(prediction, targets)

    n = len(dl)
    wandb.log({"val_rmse": ((mse / n) ** 0.5) * stds[VARIABLE]})
    wandb.log({"val_mae": (mae / n) * stds[VARIABLE]})


def train_one_batch(model, optimizer, criterion, batch, batch_idx):
    optimizer.zero_grad()

    inputs, targets = batch

    prediction = model(inputs).squeeze()
    loss = criterion(prediction, targets)

    loss.backward()
    optimizer.step()

    if batch_idx % 100 == 0:
        with torch.no_grad():
            rmse = (torch.nn.functional.mse_loss(prediction, targets) ** 0.5) * stds[VARIABLE]
            wandb.log({"train_loss": loss})
            wandb.log({"train_rmse": rmse})


def train_one_epoch(model, optimizer, criterion, epoch, dl):
    data = iter(dl)
    for i in range(len(dl)):
        train_one_batch(model, optimizer, criterion, next(data), i)


def main():
    wandb.init(project=f"MERRA2-{VARIABLE}", dir="wandb-local")

    learning_rate = wandb.config.learning_rate
    batch_size = wandb.config.batch_size
    layers = wandb.config.layers

    activation = wandb.config.activation
    activation = getattr(torch.nn, activation)

    train_dl = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=10)
    validation_dl = DataLoader(validation, batch_size=batch_size, shuffle=False, num_workers=10)

    model = get_dense_model(INPUT_SIZE, layers, OUTPUT_SIZE, activation)
    model = model.to(DEVICE)
    print(model)

    criterion = LOSS_FUNC()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    wandb.watch(model, log_freq=100)
    model.train()

    for ep in tqdm(range(EPOCHS)):
        print(ep, end=" ")
        train_one_epoch(model, optimizer, criterion, ep, train_dl)
        evaluate_one_epoch(model, ep, validation_dl)

    wandb.finish()

In [8]:
sweep_configuration = {
    "method": "bayes",
    "name": f"sweep-{DATASET}",
    "metric": {
        "goal": "minimize",
        "name": "val_rmse"
    },
    "parameters": {
        "batch_size": {"values": [2048, 4096, 8192]},
        "learning_rate": {"max": 0.1, "min": 0.0001},
        "layers": {"values": [(128, 64), (64, 128), (256, 128), (128, 256), (512, 256), (1024, 512), (512, 1024),
                              (2048, 1024), (2048, 1024), (576, 361, 36), (36, 361, 576), (512, 1024, 32),
                              (1024, 512, 128), (512, 256, 128, 64)]},
        "estimate_quantile": {"value": ESTIMATE_QUANTILE},
        "dataset": {"value": DATASET},
        "epochs": {"value": EPOCHS},
        "activation": {"values": ["ReLU", "Tanh", "PReLU", "LeakyReLU"]},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5,
    }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project=f"MERRA2-{VARIABLE}")
wandb.agent(sweep_id, function=main)

wandb: Network error (ReadTimeout), entering retry loop.


Create sweep with ID: 2r9734vo
Sweep URL: https://wandb.ai/bhavye-mathur/MERRA2-U/sweeps/2r9734vo


wandb: Agent Starting Run: 3pd951yl with config:
wandb: 	activation: LeakyReLU
wandb: 	batch_size: 8192
wandb: 	dataset: NGCTAUAV
wandb: 	epochs: 30
wandb: 	estimate_quantile: 0.9935
wandb: 	layers: [64, 128]
wandb: 	learning_rate: 0.0654268663174165
wandb: Currently logged in as: bhavye-mathur. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Path wandb-local/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb-local/wandb/ wasn't writable, using system temp directory


Sequential(
  (0): Linear(in_features=15, out_features=64, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=64, out_features=128, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=128, out_features=1, bias=True)
)


  0%|          | 0/30 [00:00<?, ?it/s]

...

wandb: Agent Starting Run: ehn8utse with config:
wandb: 	activation: LeakyReLU
wandb: 	batch_size: 4096
wandb: 	dataset: NGCTAUAV
wandb: 	epochs: 30
wandb: 	estimate_quantile: 0.9935
wandb: 	layers: [128, 64]
wandb: 	learning_rate: 0.027027414705634908
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Path wandb-local/wandb/ wasn't writable, using system temp directory


Sequential(
  (0): Linear(in_features=15, out_features=128, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=64, out_features=1, bias=True)
)


  0%|          | 0/30 [00:00<?, ?it/s]

0 1 

wandb: Ctrl + C detected. Stopping sweep.


In [14]:
def test(model, dl):
    model.eval()

    mse = 0
    mae = 0

    with torch.no_grad():
        for inputs, targets in tqdm(dl):
            prediction = model(inputs).squeeze()

            mse += torch.nn.functional.mse_loss(prediction, targets)
            mae += torch.nn.functional.l1_loss(prediction, targets)

    return (mse / len(dl)) ** 0.5 * stds[VARIABLE], (mae / len(dl)) * stds[VARIABLE]


test_dl = DataLoader(test, batch_size=2048, shuffle=False)
test_rmse, test_mae = test(model, test)

print(f"RMSE: {test_rmse} m/s")
print(f"MAE:  {test_mae} m/s")


  0%|          | 0/973 [00:00<?, ?it/s]

RMSE: 0.4102303087711334 m/s
MAE:  0.29246410727500916 m/s
